# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [29]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [30]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Sex'] = df['Sex'].fillna('None')
mean_df = df.groupby(['Sex'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Sex'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Fare'].median().reset_index()
max_df = df.groupby(['Sex'])['Fare'].max().reset_index()
min_df = df.groupby(['Sex'])['Fare'].min().reset_index()
count_df = df.groupby(['Sex'])['Fare'].agg(['count']).reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
temp = pd.merge(temp, median_df, how='left', on=['Sex'])
temp = pd.merge(temp, max_df, how='left', on=['Sex'])
temp = pd.merge(temp, min_df, how='left', on=['Sex'])
temp = pd.merge(temp, count_df, how='left', on=['Sex'])
temp.columns = ['Sex', 'Sex_Fare_Mean', 'Sex_Fare_Mode', 'Sex_Fare_Median', 'Sex_Fare_Max','Sex_Fare_Min','Sex_Fare_Count']
temp

df['Age'] = df['Age'].fillna('None')
mean_df = df.groupby(['Age'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Age'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Age'])['Fare'].median().reset_index()
max_df = df.groupby(['Age'])['Fare'].max().reset_index()
min_df = df.groupby(['Age'])['Fare'].min().reset_index()
count_df = df.groupby(['Age'])['Fare'].agg(['count']).reset_index()
temp2 = pd.merge(mean_df, mode_df, how='left', on=['Age'])
temp2 = pd.merge(temp2, median_df, how='left', on=['Age'])
temp2 = pd.merge(temp2, max_df, how='left', on=['Age'])
temp2 = pd.merge(temp2, min_df, how='left', on=['Age'])
temp2 = pd.merge(temp2, count_df, how='left', on=['Age'])
temp2.columns = ['Age', 'Age_Fare_Mean', 'Age_Fare_Mode', 'Age_Fare_Median', 'Age_Fare_Max','Age_Fare_Min','Age_Fare_Count']
temp2

df = pd.merge(df, temp, how='left', on=['Sex'])
df = df.drop(['Sex'] , axis=1)

df = pd.merge(df, temp2, how='left', on=['Age'])
df = df.drop(['Age'] , axis=1)
df.head()

,Pclass,Name,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Fare_Mean,Sex_Fare_Mode,Sex_Fare_Median,Sex_Fare_Max,Sex_Fare_Min,Sex_Fare_Count,Age_Fare_Mean,Age_Fare_Mode,Age_Fare_Median,Age_Fare_Max,Age_Fare_Min,Age_Fare_Count
0,3,"Braund, Mr. Owen Harris",1,0,A/5 21171,7.2500,NaN,S,25.523893,8.05,10.5,512.3292,0.00,577,25.504781,7.2500,7.89580,151.5500,7.125,27
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0,PC 17599,71.2833,C85,C,44.479818,7.75,23.0,512.3292,6.75,314,62.751509,0.0000,31.38750,227.5250,0.000,11
2,3,"Heikkinen, Miss. Laina",0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,7.75,23.0,512.3292,6.75,314,19.086806,7.8542,9.58125,78.8500,7.775,18
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0,113803,53.1000,C123,S,44.479818,7.75,23.0,512.3292,6.75,314,89.312500,8.0500,26.14375,512.3292,7.050,18
4,3,"Allen, Mr. William Henry",0,0,373450,8.0500,NaN,S,25.523893,8.05,10.5,512.3292,0.00,577,89.312500,8.0500,26.14375,512.3292,7.050,18


In [31]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

16 Numeric Features : ['Pclass', 'SibSp', 'Parch', 'Fare', 'Sex_Fare_Mean', 'Sex_Fare_Mode', 'Sex_Fare_Median', 'Sex_Fare_Max', 'Sex_Fare_Min', 'Sex_Fare_Count', 'Age_Fare_Mean', 'Age_Fare_Mode', 'Age_Fare_Median', 'Age_Fare_Max', 'Age_Fare_Min', 'Age_Fare_Count']



,Pclass,SibSp,Parch,Fare,Sex_Fare_Mean,Sex_Fare_Mode,Sex_Fare_Median,Sex_Fare_Max,Sex_Fare_Min,Sex_Fare_Count,Age_Fare_Mean,Age_Fare_Mode,Age_Fare_Median,Age_Fare_Max,Age_Fare_Min,Age_Fare_Count
0,3,1,0,7.2500,25.523893,8.05,10.5,512.3292,0.00,577,25.504781,7.2500,7.89580,151.5500,7.125,27
1,1,1,0,71.2833,44.479818,7.75,23.0,512.3292,6.75,314,62.751509,0.0000,31.38750,227.5250,0.000,11
2,3,0,0,7.9250,44.479818,7.75,23.0,512.3292,6.75,314,19.086806,7.8542,9.58125,78.8500,7.775,18
3,1,1,0,53.1000,44.479818,7.75,23.0,512.3292,6.75,314,89.312500,8.0500,26.14375,512.3292,7.050,18
4,3,0,0,8.0500,25.523893,8.05,10.5,512.3292,0.00,577,89.312500,8.0500,26.14375,512.3292,7.050,18


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [34]:
# 原始特徵 + 邏輯斯迴歸
df_minus = df.drop(['Age_Fare_Mean', 'Age_Fare_Mode', 'Age_Fare_Median', 'Age_Fare_Max','Age_Fare_Min','Age_Fare_Count','Sex_Fare_Mean', 'Sex_Fare_Mode', 'Sex_Fare_Median', 'Sex_Fare_Max','Sex_Fare_Min','Sex_Fare_Count'] , axis=1)

train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
print('Origin Feature / LogisticRegression : ',cross_val_score(estimator, train_X, train_Y, cv=5).mean())

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
print('Origin Feature / LinearRegression : ',cross_val_score(estimator, train_X, train_Y, cv=5).mean())

Origin Feature / LogisticRegression :  0.6847622178554621
Origin Feature / LinearRegression :  0.10444159488459337


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver t

In [38]:
# 新特徵 + 邏輯斯迴歸
print(df)
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
print('New Feature / LogisticRegression : ',cross_val_score(estimator, train_X, train_Y, cv=5).mean())



     Pclass  SibSp  Parch      Fare  Sex_Fare_Mean  Sex_Fare_Mode  \
0         3      1      0    7.2500      25.523893           8.05   
1         1      1      0   71.2833      44.479818           7.75   
2         3      0      0    7.9250      44.479818           7.75   
3         1      1      0   53.1000      44.479818           7.75   
4         3      0      0    8.0500      25.523893           8.05   
5         3      0      0    8.4583      25.523893           8.05   
6         1      0      0   51.8625      25.523893           8.05   
7         3      3      1   21.0750      25.523893           8.05   
8         3      0      2   11.1333      44.479818           7.75   
9         2      1      0   30.0708      44.479818           7.75   
10        3      1      1   16.7000      44.479818           7.75   
11        1      0      0   26.5500      44.479818           7.75   
12        3      0      0    8.0500      25.523893           8.05   
13        3      1      5   31.275

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver t